# Evaluate Results

## Correct Weighted f1

In [2]:
from decision_trees_run import ClassifierDecisionTree
from decision_trees_run import NeuronPredictorArgs
from decision_trees_run import NeuronPredictor
from decision_trees_run import NeuronPredictorEvaluator

from tqdm import tqdm
import numpy as np
import pandas as pd
import os

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [3]:
# load datasets
big_datasets = []
for layer in tqdm(range(8)):
    df = pd.read_csv(f"data/neuron_datasets/big_argmax_train_L{layer}.csv")
    big_datasets.append(df)

100%|██████████| 8/8 [04:37<00:00, 34.71s/it]


In [4]:
# load datasets
big_datasets_test = []
for layer in tqdm(range(8)):
    df = pd.read_csv(f"data/neuron_datasets/big_argmax_test_L{layer}.csv")
    big_datasets_test.append(df)

100%|██████████| 8/8 [02:18<00:00, 17.28s/it]


In [5]:
df = pd.read_csv("results_decision_tree_classifier_L0.csv")
df

,Unnamed: 0,layer,neuron,min_impurity_decrease,f1,f1_train,weighted_f1,weighted_f1_train,variable_count,rules
0,0,0,0,0.0001,0.356914,0.405063,0.609197,0.490317,129,['G6 not empty G4 flipped H6 not mine H7 empty...
1,1,0,0,0.0005,0.233240,0.262893,0.419272,0.328931,29,['G6 not empty G4 not flipped G6 not flipped G...
2,2,0,1,0.0001,0.899933,0.905306,0.993183,0.949371,149,['H7 not yours C2 not empty H5 not empty H6 em...
3,3,0,1,0.0005,0.884474,0.886870,0.990750,0.943730,81,['H7 not yours C2 not empty H5 not empty H6 em...
4,4,0,2,0.0001,0.547855,0.617696,0.754000,0.819484,13,['D6 placed B6 not empty C6 not mine D7 not mi...
...,...,...,...,...,...,...,...,...,...,...
4091,4091,0,2045,0.0005,0.695703,0.708479,0.940176,1.071423,19,"['D0 flipped => 0.7288', 'D0 not flipped B0 pl..."
4092,4092,0,2046,0.0001,0.616030,0.639805,0.859470,0.787062,459,['E6 empty E4 not flipped D6 empty C2 not empt...
4093,4093,0,2046,0.0005,0.513332,0.520567,0.762314,0.692064,91,['E6 empty E4 not flipped D6 not empty H6 plac...
4094,4094,0,2047,0.0001,0.698864,0.736402,0.839056,0.930286,21,['F2 placed B2 empty F3 empty E2 not empty C2 ...


In [8]:
min_impurity_decreases = [0.0001, 0.0005]
for layer in tqdm(range(8)):
    df = pd.read_csv(f"results_decision_tree_classifier_L{layer}.csv")
    # remove column Unnamed: 0
    df = df.drop(columns=["Unnamed: 0"])
    for neuron in range(2048):
        args = NeuronPredictorArgs(
            dataset_kind = "big_argmax",
            neuron_predictor_type = "decision_tree_classifier",
            preprocess_train_type = "round",
            preprocess_eval_type = "normal",
            layer = layer,
            neuron = neuron,
            len_data = 100000,
            dataset_train = big_datasets[layer],
            dataset_test = big_datasets_test[layer],
        )
        evaluator_round = NeuronPredictorEvaluator(args)
        # weighted_f1, f1, variable_count, rules, y_pred_test = evaluator_round.train_and_evaluate(min_impurity_decrease=0.00001)
        for j, min_impurity_decrease in enumerate(min_impurity_decreases):
            dt = ClassifierDecisionTree(layer, neuron)
            dt.load(layer, neuron, name = f"neuron_classifier_L{layer}_N{neuron}_{min_impurity_decrease}")
            evaluator_round.neuron_predictor = dt
            weighted_f1, weighted_f1_train, f1, f1_train, variable_count, rules = evaluator_round.evaluate()
            correct_row = (df["neuron"] == neuron) & (df["min_impurity_decrease"] == min_impurity_decrease)
            df.loc[correct_row, "f1_train"] = f1_train
            df.loc[correct_row, "weighted_f1_train"] = weighted_f1_train
            # df.loc[correct_row, "rules"] = rules

    # save df with new path
    df.to_csv(f"results_decision_tree_classifier_L{layer}_correct_weighted_f1.csv")

  0%|          | 0/8 [00:00<?, ?it/s]/hpi/fs00/home/jim.maar/interpretability/decision_trees_run.py:266: RuntimeWarning: divide by zero encountered in divide
  score = y / sum_y * len_y
/hpi/fs00/home/jim.maar/interpretability/decision_trees_run.py:266: RuntimeWarning: invalid value encountered in multiply
  score = y / sum_y * len_y
/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/hpi/fs00/home/jim.maar/interpretability/decision_trees_run.py:266: RuntimeWarning: divide by zero encountered in divide
  score = y / sum_y * len_y
/hpi/fs00/home/jim.maar/interpretability/decision_trees_run.py:266: RuntimeWarning: invalid value encountered in multiply
  score = y / sum_y * len_y
/hpi/fs0

KeyboardInterrupt: 

## Evaluate Results

### Create Table
- Remove expamples with 0.0000 F1 (Not really remove but don't count in the score ...)
- TODO: Investigate what exactly happens, when f1 is 0.0: I think the DT does not find a single split with the correct impurity decrease, then the dt predicts NONE ...
- Layer, min_impurity_decrease, Avg F1, Avg F1_train, Avg weighted_F1, Avg weighted_F1_train, Avg Variable Count, Fraction of 0.0000

In [ ]:
# Remove expamples with 0.0000 F1 (Not really remove but don't count in the score ...)
# TODO: Investigate what exactly happens, when f1 is 0.0: I think the DT does not find a single split with the correct impurity decrease, then the dt predicts NONE ...
# Layer, min_impurity_decrease, Avg F1, Avg F1_train, Avg weighted_F1, Avg weighted_F1_train, Avg Variable Count, Fraction of 0.0000

for layer in range(8):
    df = pd.read_csv(f"results_decision_tree_classifier_L{layer}_correct_weighted_f1.csv")
    for min_impurity_decrease in min_impurity_decreases:
        df_small = df[df["min_impurity_decrease"] == min_impurity_decrease]
        df_real = df_small[df_small["f1"] != 0]
        f1 = df_real["f1"].mean()
        f1_train = df_real["f1_train"].mean()
        weighted_f1 = df_real["weighted_f1"].mean()
        weighted_f1_train = df_real["weighted_f1_train"].mean()
        variable_count = df_small["variable_count"].mean()
        fraction_0 = np.mean(f1 == 0)